In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("Train.csv1")


In [3]:
X= train.drop("AveragePrice", axis = 1)
y = train["AveragePrice"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Gradient Boost

In [5]:
from sklearn.ensemble import GradientBoostingRegressor

In [6]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [7]:
y_pred = model.predict(X_test)

In [8]:
model.score(X_train, y_train)

0.7570679028716399

In [9]:
model.score(X_test, y_test)

0.7333009029759343

In [10]:
#mean squared error
metrics.mean_squared_error(y_test, y_pred)

0.029043725521669353

The accuracy is very poor for Gradient boost

# XG boost

In [11]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

xgr = XGBRegressor()

xgr.fit(X_train, y_train)

C:\Users\Md Khalid Siddique\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:10:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [12]:
y_pred_xgr = xgr.predict(X_test)

In [13]:
xgr.score(X_train,y_train)

0.7565228359735723

In [14]:
xgr.score(X_test,y_test)

0.7327098528577826

In [15]:
#mean squared error
metrics.mean_squared_error(y_test, y_pred_xgr)

0.02910809130915307

Again a poor accuracy, lets try out another algorithm

# Random Forest Regressor

In [16]:
from sklearn.ensemble import RandomForestRegressor 

In [17]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

C:\Users\Md Khalid Siddique\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [18]:
y_pred_Rf = model.predict(X_test)

In [19]:
#Performance of model on train data
model.score(X_train, y_train)

0.9772357436932319

In [20]:
model.score(X_test, y_test)

0.8714713813010273

In [21]:
#mean squared error
metrics.mean_squared_error(y_test, y_pred_Rf)

0.01399686

Comparing to previous scores, the score of Random forest is impressive. So i will tune the parameters to improve its performance

# Hyperparameter tuning for Random Forest

In [22]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 

C:\Users\Md Khalid Siddique\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [23]:
#Declare data preprocessing steps
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

In [24]:
#Declare hyperparameters to tune
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

In [25]:
#Tune model using cross-validation pipeline
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('randomforestregressor',
                                        RandomForestRegressor(bootstrap=True,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              min_impurity_decrease=0.0,
                                                              min_impurity_split

In [26]:
#Perfomance on train data
clf.score(X_train, y_train)

0.9850350707672907

In [27]:
#MSE
pred = clf.predict(X_test)
mean_squared_error(y_test, pred)

0.01231395370208332

In [28]:
#perfomance on testv data
r2_score(y_test, pred) 

0.8869249631666052

After tuning our model score has improved to 0.89 with mse of 0.0116

# Extra Trees Regressor

In [29]:
from sklearn.ensemble import ExtraTreesRegressor

In [30]:
reg = ExtraTreesRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)

In [31]:
reg.score(X_train, y_train)

1.0

In [32]:
reg.score(X_test, y_test)

0.922454686818913

In [33]:
pred = reg.predict(X_test)
metrics.mean_squared_error(y_test, pred)

0.00844474097083333

Extra trees regressor is performing better than Random forest with an accuracy of 0.92

# Hyperparameter tuning for ExtraTreesRegressor

In [34]:
#Declare data preprocessing steps
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         ExtraTreesRegressor(n_estimators=100))

In [35]:
#Declare hyperparameters to tune
hyperparameters = { 'extratreesregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'extratreesregressor__max_depth': [None, 5, 3, 1]}

In [36]:
#Tune model using cross-validation pipeline
cle= GridSearchCV(pipeline, hyperparameters, cv=10)
cle.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('extratreesregressor',
                                        ExtraTreesRegressor(bootstrap=False,
                                                            criterion='mse',
                                                            max_depth=None,
                                                            max_features='auto',
                                                            max_leaf_nodes=None,
                                                            min_impurity_decrease=0.0,
                                                            min_impurity_split=None,
        

In [37]:
#Perfomance on train data
cle.score(X_train, y_train)

1.0

In [38]:
pred = cle.predict(X_test)
metrics.mean_squared_error(y_test, pred)

0.008543755037499991

In [39]:
cle.score(X_test, y_test)

0.9215454728080252

Here the accuracy of model went down after tuning, hence will go with the model with out tuning

# Test dataset

Now lets import our test dataset and get the average_price

In [40]:
test_1 = pd.read_csv("Test.csv2")

In [41]:
test_1.head()

,Total_volume,Small_Hass,Large_Hass,Extra_large_hass,Total_bags,Small_bags,Large_bags,Xlarge_bags,type,year,...,Seattle,SouthCarolina,SouthCentral,Southeast,Spokane,StLouis,Syracuse,Tampa,TotalUS,West
0,13045.36,7163.20,5530.28,0.00,1523.516,610.864,0.00,0.0,1,2015,...,0,0,0,0,0,0,0,0,0,0
1,22282.57,12876.46,2651.64,8.06,6746.410,6538.450,207.96,0.0,1,2016,...,0,0,0,0,0,0,0,0,0,0
2,435990.48,235664.26,141949.05,14634.32,43742.850,37689.450,6053.40,0.0,0,2015,...,0,0,0,0,0,0,0,0,0,0
3,219089.52,43010.81,111487.02,218.09,64373.600,63388.040,985.56,0.0,0,2016,...,0,0,0,0,0,0,0,0,0,0
4,7431.86,666.13,3284.48,14.18,3467.070,3467.070,0.00,0.0,1,2016,...,0,0,0,0,0,0,0,0,0,0


In [42]:
len(test_1)

2249

In [43]:
import numpy as np

In [44]:
test_1 = np.nan_to_num(test_1)

In [45]:
p =  reg.predict(test_1)
predicted_average = p.tolist()
df = pd.DataFrame(predicted_average)
df.to_csv("Average_price.csv", index = False)


In [46]:
a = pd.read_csv("Average_price.csv")
a.head()

,0
0,1.5712
1,1.4183
2,1.0502
3,1.4908
4,1.8664


In [47]:
#Predicted Average price
t = a.rename(columns={'0': 'Average_price'})
t.head()

,Average_price
0,1.5712
1,1.4183
2,1.0502
3,1.4908
4,1.8664


# Flask Api

In [49]:
import pickle
with open("Avocado Price Prediction.pkl", 'wb') as f:
    pickle.dump(reg,f)

In [50]:
from flask import Flask,request,jsonify,render_template
app=Flask(__name__)
model=pickle.load(open('Avocado Price Prediction.pkl','rb'))
debug=True
@app.route('/')
def home():
    
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
   
    int_features=[int(x) for x in request.form.values()]
    final_features=[np.array(int_features)]
    prediction=model.predict(final_features)
    
    output=prediction[0]
    
   
    return render_template('index.html',prediction_text='Enjoy the Taste {}'.format(output))

if __name__=='__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2020 21:30:01] "GET / HTTP/1.1" 200 -
[2020-05-25 21:30:51,021] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\Users\Md Khalid Siddique\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Md Khalid Siddique\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Md Khalid Siddique\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Md Khalid Siddique\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Md Khalid Siddique\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Md Khalid Siddique\Anaconda3\lib\site-pa